In [11]:
import numpy as np
import pandas as pd
import requests
from zipfile import ZipFile

In [6]:
URL = 'https://localhost:3000/restaurants/create-restaurant'

In [18]:
FILE = "./italian_restaurants.zip"
zip_file = ZipFile(FILE)
zip_file

['italian_restaurants.csv', '__MACOSX/._italian_restaurants.csv']


FileNotFoundError: [Errno 2] No such file or directory: './italian_restaurants.csv'

In [14]:
data

,Unnamed: 0,restaurant_link,restaurant_name,original_location,country,region,province,city,address,latitude,...,excellent,very_good,average,poor,terrible,food,service,value,atmosphere,keywords
0,650043,g10029467-d18338571,L'Ortal,"[""Europe"", ""Italy"", ""Friuli Venezia Giulia"", ""...",Italy,Friuli Venezia Giulia,Province of Pordenone,NaN,"Via Jacopo Ortis 15, 33090, Vito d'Asio Italy",46.228010,...,4.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,650044,g10029467-d23200955,Osteria con Cucina Lorenzini,"[""Europe"", ""Italy"", ""Friuli Venezia Giulia"", ""...",Italy,Friuli Venezia Giulia,Province of Pordenone,NaN,"Via Chiamp, 18, 33090, Vito d'Asio Italy",46.264570,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,650045,g10029597-d6878487,Ristorante Da Renzo,"[""Europe"", ""Italy"", ""Friuli Venezia Giulia"", ""...",Italy,Friuli Venezia Giulia,Province of Pordenone,NaN,"Via Galantz 2, 33090 San Francesco, Vito d'Asi...",46.312300,...,2.0,1.0,0.0,0.0,0.0,4.5,4.0,4.5,NaN,NaN
3,650046,g10038815-d14177798,Pizza Point,"[""Europe"", ""Italy"", ""Trentino-Alto Adige"", ""Pr...",Italy,Trentino-Alto Adige,Province of South Tyrol,NaN,"Via del Borgo 4B, 39030 Valdaora di Sopra, Val...",46.758360,...,4.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,650047,g10045458-d12940567,Pizzeria Snoopy 2,"[""Europe"", ""Italy"", ""Emilia-Romagna"", ""Provinc...",Italy,Emilia-Romagna,Province of Bologna,NaN,"Piazza Liberta' 14, 40060 Toscanella, Dozza Italy",44.383064,...,6.0,1.0,1.0,0.0,5.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224758,874801,g9885748-d12879403,La Locanda dell'Eremita,"[""Europe"", ""Italy"", ""Campania"", ""Province of C...",Italy,Campania,Province of Caserta,NaN,"Via San Vitaliano 30, 81100 Casola, Caserta Italy",41.094303,...,4.0,1.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
224759,874802,g9885748-d4002177,Ristorante Pizzeria La Fenice,"[""Europe"", ""Italy"", ""Campania"", ""Province of C...",Italy,Campania,Province of Caserta,NaN,"Via Dottore Ferdinando Rossi 41, 81100 Casola,...",41.094723,...,73.0,76.0,24.0,10.0,7.0,4.0,4.0,4.0,4.0,NaN
224760,874803,g9885748-d6905025,Agriturismo l'Eremo,"[""Europe"", ""Italy"", ""Campania"", ""Province of C...",Italy,Campania,Province of Caserta,NaN,"Via San Vitaliano 34, 81100 Casola, Caserta Italy",41.094180,...,53.0,42.0,16.0,4.0,2.0,4.0,4.5,4.5,NaN,NaN
224761,874804,g9885970-d15265965,L'Etoile,"[""Europe"", ""Italy"", ""Friuli Venezia Giulia"", ""...",Italy,Friuli Venezia Giulia,Province of Pordenone,NaN,"Viale San Giorgio 9, 33081 Castello d'Aviano, ...",46.069830,...,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
